# Trigrams in Bilinear Transformers


In this document, we will analyze what transformer do in their first attention layer. Which is closely related with trigrams

---

## Setup

We will take a very naive approach; perform the diagonal approach where the same token is inserted into the value component of a head (virtual) and one that follows the residual stream (direct). Given some background knowledge, the setup is actually quite simple; we study the following matrix. I won't go into depth on this math here.

$$ U_{oq} P_{qa} (E_{jb} B_{aij} O_{il} V_{lk} E_{kb}) $$

## Usefulness

While the results are quite striking, I'm unsure of the generality of this technique. This is only useful to study same-token heads, which actually often occurs in the first layer but should be taken with a pinch of salt.

## Code

In [ ]:
from shared.transformer import Transformer, Config
import plotly.express as px
from shared.tensors import *
import torch
import pandas as pd
from einops import *

In [ ]:
torch.set_grad_enabled(False)
pd.set_option('display.max_columns', None)

name = "tdooms/TinyStories-2-256"
config = Config.from_pretrained(name)
model = Transformer.from_pretrained(name, config=config).cuda()
vocab = model.vocab

_ = model.center_unembed().fold_norms()

In [ ]:
def attention_features(layer, head, mlp=0):
    diag = einsum(
        model.w_e, model.w_e, model.ov[layer, head], model.w_l[mlp], model.w_r[mlp], model.w_p[mlp], model.w_u,
        "emb1 i, emb2 i, ov emb1, hid ov, hid emb2, res hid, out res -> out i"
    ).cpu()
    
    o, s, q = torch.svd(diag)
    # px.line(s[:64].cpu()).show()
    
    df = vocab.get_max_activations(diag.T, ["virtual", "direct"], 10)

    for i in range(0, 10):
        tops = (o[:, i:i+1] @ torch.diag(s[i:i+1]) @ q.T[i:i+1])
        df = df.join(vocab.get_max_activations(tops.T, [f"virtual_{i}", f"direct_{i}"], 10, val_name=f"value_{i}"))

    return df

## Results
The results are large pandas dataframes. They should be read as (input, output, value) triplets, where the input is a previous token that this specific attention head may look at. The value is simple how strong the connection is, there is no clear explanation for this value except higher is better.

The first triplet is an analysis on the whole diagonal, the following ones are the svd components.

Let's first look at attention head 0.0

In [ ]:
attention_features(layer=0, head=0).round(2)

At first, this may seem a bit random. There is a lot of structure but what is it?

Looking at this from a trigram perspective makes a lot of sense. For instance, in component 2, it's easy to see that ``no (happy) faces`` or ``no (loud) bark`` make sense.

For the last component, maybe all ``girls (sno) re``.

Exactly determining these trigrams won't be covered here.

In [ ]:
attention_features(layer=0, head=1)

This head is slightly strange, first of all, I like to call this "the mommy head". But when looking closer, it does a lot more.

The first two components, I don't really understand, why are ``res`` and ``t`` useful? They don't compose in rest because: first, ``rest`` itself is a token, second, otherwise it should be ``##res``. The 3rd component (index 2) is something about negative adjectives. The fifth is about negation, the sixth is maybe about trigrams of ``couldn (') t`` (accents are tokenized separately, always). The seventh is obviously the mommy one.

Anyway, from some further analysis (not shown), this seems to be a head that has triplets closely related to verbs and subjects.

In [ ]:
attention_features(layer=0, head=2)

I won't go too in-depth from now on, however, this has a funny animal component for some reason. Not sure what ``##ned`` is though.

In [ ]:
attention_features(layer=0, head=3)

Again, some cool structure, somewhat related to quotes it seems.